### General
##### General Pipeline
1) Wget the right gtf and fasta ref genome files for the Drosophila (or whatever organism) genome into appropriate directories

2) Then parse GTF file for start/end coordinates for each gene. ATAC-seq data: split this interval into non-overlapping windows of x bp and assign peak counts. RNA-seq data: find TSS of each gene and do +/- specified x bases for each one. Then output BED file(s) delimited by start, stop coords you just made + the label. 

3) Next, run getfasta on the BED file(s) you just made. This will output a bunch of fasta files with just the gene_ids + actual sequences for each window interval you defined. 

4) Finally, run a script to concatenate together the outputted fasta files from getfasta and your bed files. The outputted CSV file will have the sequences themselves (rather than just start/stop coords) for each window and their corresponding label.


### Preprocess RNA-seq data 

Run scripts to preprocess RNA-seq data (in fasta file format) into BED files with sequences of +/- x bp windows up and downstream of the TSS respectively (2x bp overall). Discretize counts, run getfasta to get corresponding sequences, and then merge into final CSV. This following script is for excel spreadsheets of log-normalized counts. 

In [ ]:
!python3 python/rna_bed.py -gtf_filepath '../0-data/0-refs/agingeye/dmel-all-r6.55.gtf' -counts_filepath '../0-data/1-experimental/counts/agingeye/counts.xlsx' -sequence_length 600

This following is for text count data that is NOT in an excel spreadsheet - you just parse it in 1 by 1 and do a bunch of manual operations to get your desired dataset. 

In [14]:
!python3 python/rna_bed_default.py -gtf_filepath '../0-data/0-refs/agingeye/dmel-all-r6.55.gtf' -counts_filepath '../0-data/1-experimental/counts/agingeye/cts.txt' -sequence_length 600

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
/Users/nicolaskim/anaconda3/envs/dlenv/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


This code runs getFasta on the files I just generated into bed format in the appropriate directory, using the appropriate reference genome (r6.55). 

In [15]:
!./bash/rna_getfasta.sh '../0-data/0-refs/agingeye/dmel-all-chromosome-r6.55.fasta'

This code converts the outputted fasta files from the getfasta script above into CSV format for processing. Displays first three rows of output file. Some filepaths I used for the bed files are the following: '../0-data/1-experimental/bed/agingeye/cts_bed.bed', '../0-data/1-experimental/bed/agingeye/D60vsD10.bed'. Some filepaths I used for the getfasta output are the following: '../0-data/2-final/0-getfasta/agingeye/cts_bed.out.fa', '../0-data/2-final/0-getfasta/agingeye/D60vsD10.out.fa'

In [16]:
!python3 python/rna_getfasta.py -input_bed_file '../0-data/1-experimental/bed/agingeye/cts_bed.bed' -input_fasta_file '../0-data/2-final/0-getfasta/agingeye/cts_bed.out.fa' -output_csv_file '../0-data/2-final/1-finalcsv/agingeye/' -output_csv_file_identifier 'cts_bed'

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
First three rows:                                              sequence  count
0  AAGCATAAATGGGTGATCTTATATCACGCTCACTTGAGACATTTAT...      1
1  TGAACCCAAATTATTATGATACAATAATATTTAAATGAATTGTTCC...      1
2  CTATACGAAGGAGACTGAGTGCTCCATTTGTCTAATGCTAATTGTG...      1
Number of ones in whole dataset 20088
umber of zeros in whole dataset 10499
Number of ones in train dataset 16102
Number of zeros in train dataset 8367
Number of ones in test dataset 3986
Number of zeros in test dataset 2132


### Preprocess ATAC-seq data
Run scripts to preprocess ATAC-seq data into non-overlapping windows of 600 bp each. Run bedtools getfasta to get the actual corresponding sequence for each window, using the proper reference genome. Note that the testing directory used DNAase-seq data, not ATAC-seq data, and used the hg19 grch37 reference genome (DNA) instead of what was needed for this project specifically. 

In [ ]:

!python3 python/atac_bed.py -input_bed_directory '../0-data/1-experimental/bed/encode/raw/' -output_bed_directory '../0-data/1-experimental/bed/encode/binned/' -max_overlap 200 -sequence_length 600

Outputted files will be not in fasta format, but instead in tab delimited format since I wanted an easier way of labeling the sequences with appropriate counts.

In [23]:
!./bash/atac_getfasta.sh '../0-data/0-refs/encode/hg19.fa'

Now process the outputted fasta file into CSV file format

In [ ]:
!python3 python/atac_getfasta.py -input_bed_file '../0-data/1-experimental/bed/encode/binned/E003.bed' -input_fasta_file '../0-data/2-final/0-getfasta/encode/E003.out.fa' -output_csv_file '../0-data/2-final/1-finalcsv/E003'